In [ ]:
import cv2
import numpy as np

def detect_center_line(frame):
    """Detekcija linije pomoću Canny i Hough transformacije."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=200, maxLineGap=20)

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            return (x1, y1, x2, y2)

    return None

def detect_blue_objects(frame, lower_blue, upper_blue):
    """Detekcija plavih objekata na osnovu HSV opsega."""
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Pronađi konture plavih regiona
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected_objects = []
    for cnt in contours:
        if 500 < cv2.contourArea(cnt):  # Filtriraj male konture
            x, y, w, h = cv2.boundingRect(cnt)
            detected_objects.append((x, y, w, h))

    return detected_objects

def detect_cross(x, y, k, n):
    """Provera da li objekat prelazi liniju."""
    yy = k * x + n
    return -1 <= (yy - y) <= 1

def get_line_params(line_coords):
    """Izračunavanje parametara linije y = kx + n."""
    x1, y1, x2, y2 = line_coords
    k = (y2 - y1) / (x2 - x1)
    n = y1 - k * x1
    return k, n

def process_frame(frame, lower_blue, upper_blue):
    """Procesiranje jednog frejma."""
    # Detekcija linije
    line_coords = detect_center_line(frame)
    if not line_coords:
        return 0  # Ako nema linije, preskoči frejm

    x1, y1, x2, y2 = line_coords
    cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Nacrtaj liniju

    k, n = get_line_params(line_coords)

    # Detekcija plavih objekata
    detected_objects = detect_blue_objects(frame, lower_blue, upper_blue)
    count = 0
    for x, y, w, h in detected_objects:
        cx, cy = x + w // 2, y + h // 2  # Centar objekta
        if detect_cross(cx, cy, k, n):
            count += 1
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    return count

def process_video(video_path):
    """Procesiranje celog videa."""
    lower_blue = np.array([100, 150, 0])  # Opseg HSV za plavu boju
    upper_blue = np.array([140, 255, 255])

    cap = cv2.VideoCapture(video_path)
    total_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        count = process_frame(frame, lower_blue, upper_blue)
        total_count += count

        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return total_count

# Primer korišćenja:
video_path = "data/video_4.mp4"  # Zamenite sa vašom putanjom do videa
total_objects = process_video(video_path)
print(f"Ukupno detektovano objekata: {total_objects}")


Ukupno detektovano objekata: 0
